In [4]:
from VAMPIRE_backend import bdreg,pca_bdreg,clusterSM

In [5]:
from __future__ import division
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import pickle
import numpy as np
from matplotlib import pyplot as plt
from natsort import natsorted
from time import time

In [6]:
from time import time
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %.3f" %(f.__name__,end-start))
        return r
    return wrapper

In [7]:
bdreg = _time(bdreg)
pca_bdreg = _time(pca_bdreg)
clusterSM = _time(clusterSM)

In [8]:
N=50 #equidistant points along cell boundary

dfsrc = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\hovernet_out\df'
outpth = os.path.join(os.path.dirname(dfsrc),'vampire_out')

In [9]:
setpaths = natsorted([_ for _ in os.listdir(dfsrc) if _.endswith('pkl')])
setIDs = [os.path.splitext(_)[0] for _ in setpaths] #name of plot?
conditions = setIDs #clear definition is needed in terms of how this will be used later
setpaths = [os.path.join(dfsrc,_) for _ in setpaths]

In [10]:
# iterate dfs to compile big df
# pkls = pd.read_pickle('2d_skin_hovernet.pkl')
start = time()
pkls = []
for setpath in setpaths:
    pkl = pd.read_pickle(setpath)
    pkls.append(pkl)
pkls = pd.concat(pkls,ignore_index=True)
print(round(time()-start),'seconds')

5 seconds


In [12]:
pkls['imID']

0         monkey_fetus_40_0301
1         monkey_fetus_40_0301
2         monkey_fetus_40_0301
3         monkey_fetus_40_0301
4         monkey_fetus_40_0301
                  ...         
207941    monkey_fetus_40_0301
207942    monkey_fetus_40_0301
207943    monkey_fetus_40_0301
207944    monkey_fetus_40_0301
207945    monkey_fetus_40_0301
Name: imID, Length: 207946, dtype: object

In [15]:
start = time()
pkls.to_feather(os.path.join(dfsrc,'pickle_stack.ftr'))
print(round(time()-start),'seconds')

3 seconds


In [ ]:
comps = []
for value in pkls.imID.unique():
    numclass = 12
    a=plt.hist(pkls.loc[pkls['imID'] == value]['type'],numclass)
    comps.append(a[0]/sum(a[0])*100)

In [ ]:
compsdf = pd.DataFrame(comps)
compsdf['imID']= pkls.imID.unique()
compsdf.to_csv('2d_skin_cell_comp.csv')

In [ ]:
#build model by cell type
for clnum in range(2,9):
    modelname = 'sweat_cl{}'.format(clnum)
    idx = 6
    bd1 = pkls[pkls['type']==idx+1].reset_index()
    B=bd1['contour']
    VamModel = {
                "N": [],
                "bdrn": [],
                "mdd": [],
                "pc": [],
                "clnum": [],
                "pcnum": [],
                "mincms": [],
                "testmean": [],
                "teststd": [],
                "boxcoxlambda": [],
                "C": [],
                "Z": []
            }
    build_model = True
    bdpc, VamModel = bdreg(B, N, VamModel, build_model)
    score, VamModel = pca_bdreg(bdpc, VamModel, True)
    pcnum = None # none is 20 by default
    IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)

    f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
    pickle.dump(VamModel, f)
    f.close()

In [ ]:
#build model by cell type
tissues = ['corneum','spinosum','shaft','follicle','muscle','oil','sweat','nerve','blood','ecm','fat','white']
for clnum in range(2,9):
    for idx,tissue in enumerate(tissues):
        if tissue=='corneum':continue
        modelname = '{}_cl{}'.format(tissue,clnum)
        bd1 = pkls[pkls['type']==idx+1].reset_index()
        B=bd1['contour']
        VamModel = {
                    "N": [],
                    "bdrn": [],
                    "mdd": [],
                    "pc": [],
                    "clnum": [],
                    "pcnum": [],
                    "mincms": [],
                    "testmean": [],
                    "teststd": [],
                    "boxcoxlambda": [],
                    "C": [],
                    "Z": []
                }
        build_model = True
        bdpc, VamModel = bdreg(B, N, VamModel, build_model)
        score, VamModel = pca_bdreg(bdpc, VamModel, True)
        pcnum = None # none is 20 by default
        IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)

        f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
        pickle.dump(VamModel, f)
        f.close()

building model


In [ ]:
clnum = 5
modelname = 'corneum_cl{}'.format(clnum)
modelpth = os.path.join(*[outpth, modelname, modelname+'.pickle'])

In [ ]:
# load model
experimental = False
build_model = False
try:
    f = open(modelpth, 'rb')
except:
    print('the model does not exist. please replace model name to the one you built')
vampire_model = pickle.load(f)
N = vampire_model['N']
clnum = vampire_model['clnum']
pcnum = vampire_model['pcnum']

In [ ]:
dists=np.zeros((len(setpaths),clnum))
for idxA,(setpath,condition,setID) in enumerate(zip(setpaths,conditions,setIDs)):
    pkl = pd.read_pickle(setpath)
    pkl = pkl[pkl['contour'].map(len) > 5]
    pkl = pkl[pkl['type']==idx+1].reset_index(drop=True)
    B = pkl['contour']
    pkl = pkl.drop(columns=['contour'])
    bdpc, vampire_model = bdreg(B, N, vampire_model, build_model) #409sec
    score, vampire_model = pca_bdreg(bdpc, vampire_model, build_model)
    IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, vampire_model, False, condition, setID,modelname)
    pkl['Shape mode']= pd.Series(IDX)
    pkl.to_csv(setpath.replace('pkl','csv'), index=False)
    a=plt.hist(pkl['Shape mode'],clnum)
    dists[idxA,:] = np.array(a[0])

In [ ]:
dists2 = np.around(dists*100/dists.sum(axis=1)[:,None],decimals=2)
dists2 = pd.DataFrame(dists2)
dists2['setid']=setIDs
sumfn = 'vampire summary.csv'
sumfn = os.path.join(*[outpth, modelname,sumfn])
dists2.to_csv(sumfn)